In [4]:
from rdkit import Chem
import rdkit.Chem.AllChem as AllChem
# all the molecules can have saturated hydrogen connectivity at start
smiles_list = ["[CH:1]1[O:2][CH:3]1", "[CH3:1][NH2:4]", "[CH3:3][CH:5]=[O:6]"]
outcome = [Chem.MolFromSmiles(s) for s in smiles_list]
PLEVEL = 100

mapnums = [a.GetAtomMapNum() for m in outcome for a in m.GetAtoms() if a.GetAtomMapNum()]
if len(mapnums) != len(set(mapnums)): # duplicate?
    if PLEVEL >= 1: print('Found duplicate mapnums in product - need to stitch')
    # need to do a fancy merge
    merged_mol = Chem.RWMol(outcome[0])
    merged_map_to_id = {a.GetAtomMapNum(): a.GetIdx() for a in outcome[0].GetAtoms() if a.GetAtomMapNum()}
    for j in range(1, len(outcome)):
        new_mol = outcome[j]
        for a in new_mol.GetAtoms():
            if a.GetAtomMapNum() not in merged_map_to_id:
                merged_map_to_id[a.GetAtomMapNum()] = merged_mol.AddAtom(a)
        for b in new_mol.GetBonds():
            bi = b.GetBeginAtom().GetAtomMapNum()
            bj = b.GetEndAtom().GetAtomMapNum()
            if PLEVEL >= 10: print('stitching bond between {} and {} in stitch has chirality {}, {}'.format(
                bi, bj, b.GetStereo(), b.GetBondDir()
            ))
            if not merged_mol.GetBondBetweenAtoms(
                    merged_map_to_id[bi], merged_map_to_id[bj]):
                merged_mol.AddBond(merged_map_to_id[bi],
                    merged_map_to_id[bj], b.GetBondType())
                merged_mol.GetBondBetweenAtoms(
                    merged_map_to_id[bi], merged_map_to_id[bj]
                ).SetStereo(b.GetStereo())
                merged_mol.GetBondBetweenAtoms(
                    merged_map_to_id[bi], merged_map_to_id[bj]
                ).SetBondDir(b.GetBondDir())
    outcome = merged_mol.GetMol()
    if PLEVEL >= 1: print('Merged editable mol, converted back to real mol, {}'.format(Chem.MolToSmiles(outcome, True)))
else:
    new_outcome = outcome[0]
    for j in range(1, len(outcome)):
        new_outcome = AllChem.CombineMols(new_outcome, outcome[j])
    outcome = new_outcome

Found duplicate mapnums in product - need to stitch
stitching bond between 1 and 4 in stich has chirality STEREONONE, NONE
stitching bond between 3 and 5 in stich has chirality STEREONONE, NONE
stitching bond between 5 and 6 in stich has chirality STEREONONE, NONE
Merged editable mol, converted back to real mol, [CH:1]1([NH2:4])[O:2][CH:3]1[CH:5]=[O:6]


In [2]:
from qiskit import qasm3, QuantumCircuit, QuantumRegister, ClassicalRegister
 
# Build the circuit
qreg = QuantumRegister(3)
creg = ClassicalRegister(3)
qc = QuantumCircuit(qreg, creg)
with qc.switch(creg) as case:
    with case(0):
        qc.x(0)
    with case(1, 2):
        qc.x(1)
    with case(case.DEFAULT):
        qc.x(2)
 
# Export to an OpenQASM 3 string.
qasm_string = qasm3.dumps(qc, experimental=qasm3.ExperimentalFeatures.SWITCH_CASE_V1)
print(qasm_string)

OPENQASM 3.0;
include "stdgates.inc";
bit[3] c1;
int switch_dummy;
qubit[3] q1;
switch_dummy = c1;
switch (switch_dummy) {
  case 0: {
    x q1[0];
  }
  break;
  case 1:
  case 2: {
    x q1[1];
  }
  break;
  default: {
    x q1[2];
  }
  break;
}

